# Libraries&Function 

In [1]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from reskit.core import Transformer, Pipeliner

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def orig(x):
    return x

import copy

In [2]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict

import matrix_eig as me

# Train mosters
## Logistic Regression
### Chose the best model for each k

In [3]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig))]

normalizers = [('origN', Transformer(orig))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low0', Transformer(me.matrix_eig_0, collect=['X_vec'])),
               ('low5', Transformer(me.matrix_eig_5, collect=['X_vec'])),
               ('low10', Transformer(me.matrix_eig_10, collect=['X_vec'])),
               ('low20', Transformer(me.matrix_eig_20, collect=['X_vec'])),
               ('low30', Transformer(me.matrix_eig_30, collect=['X_vec'])),
               ('low40', Transformer(me.matrix_eig_40, collect=['X_vec'])),
               ('low50', Transformer(me.matrix_eig_50, collect=['X_vec'])),
               ('low60', Transformer(me.matrix_eig_60, collect=['X_vec'])),
               ('low70', Transformer(me.matrix_eig_70, collect=['X_vec'])),
               ('low80', Transformer(me.matrix_eig_80, collect=['X_vec'])),
               ('low90', Transformer(me.matrix_eig_90, collect=['X_vec'])),
               ('low100', Transformer(me.matrix_eig_100, collect=['X_vec'])),
               ('low110', Transformer(me.matrix_eig_110, collect=['X_vec'])),
               ('low120', Transformer(me.matrix_eig_120, collect=['X_vec'])),
               ('low130', Transformer(me.matrix_eig_130, collect=['X_vec'])),
               ('low140', Transformer(me.matrix_eig_140, collect=['X_vec'])),
               ('low150', Transformer(me.matrix_eig_150, collect=['X_vec'])),
               ('low160', Transformer(me.matrix_eig_160, collect=['X_vec'])),
               ('low170', Transformer(me.matrix_eig_170, collect=['X_vec'])),
               ('low180', Transformer(me.matrix_eig_180, collect=['X_vec'])),
               ('low190', Transformer(me.matrix_eig_190, collect=['X_vec'])),
               ('low200', Transformer(me.matrix_eig_200, collect=['X_vec'])),
               ('low210', Transformer(me.matrix_eig_210, collect=['X_vec'])),
               ('low220', Transformer(me.matrix_eig_220, collect=['X_vec'])),
               ('low230', Transformer(me.matrix_eig_230, collect=['X_vec'])),
               ('low240', Transformer(me.matrix_eig_240, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec'])),
               ('low263', Transformer(me.matrix_eig_263, collect=['X_vec']))
              ]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('SVC', SVC())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
     SVC=dict(
        C=[0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
        degree=[2, 3, 4],
        kernel=['linear', 'poly', 'rbf', 'sigmoid'],
        max_iter=[50, 100],
    )
    )

banned_combos = []

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,origF,var_threshold,minmax,SVC
1,UCLAsource,origW,origN,low0,var_threshold,minmax,SVC
2,UCLAsource,origW,origN,low5,var_threshold,minmax,SVC
3,UCLAsource,origW,origN,low10,var_threshold,minmax,SVC
4,UCLAsource,origW,origN,low20,var_threshold,minmax,SVC
5,UCLAsource,origW,origN,low30,var_threshold,minmax,SVC
6,UCLAsource,origW,origN,low40,var_threshold,minmax,SVC
7,UCLAsource,origW,origN,low50,var_threshold,minmax,SVC
8,UCLAsource,origW,origN,low60,var_threshold,minmax,SVC
9,UCLAsource,origW,origN,low70,var_threshold,minmax,SVC


In [ ]:
result = pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                          scoring=['roc_auc'],  results_file = "Results/Ver3.3/result_1.csv")

No previous results found.
Line: 1/30
Line: 2/30
Line: 3/30
Line: 4/30
Line: 5/30
Line: 6/30
Line: 7/30
Line: 8/30
Line: 9/30
Line: 10/30
Line: 11/30
Line: 12/30
Line: 14/30
Line: 15/30
Line: 16/30
Line: 17/30
Line: 18/30
Line: 19/30
Line: 20/30
Line: 21/30
Line: 22/30
Line: 23/30
Line: 24/30
Line: 25/30
Line: 26/30
Line: 27/30
Line: 28/30
Line: 29/30
Line: 30/30


In [5]:
result

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers,grid_roc_auc_mean,grid_roc_auc_std,grid_roc_auc_best_params,eval_roc_auc_mean,eval_roc_auc_std,eval_roc_auc_scores
0,UCLAsource,origW,origN,origF,var_threshold,minmax,SVC,0.594326,0.169768,"{'kernel': 'linear', 'C': 0.01, 'max_iter': 10...",0.593333,0.156162,[ 0.63333333 0.56 0.64 0.85 ...
1,UCLAsource,origW,origN,low0,var_threshold,minmax,SVC,0.543972,0.130199,"{'kernel': 'linear', 'C': 0.01, 'max_iter': 50...",0.531,0.18069,[ 0.2 0.6 0.56 0.65 0.25 0.55 0.85 0....
2,UCLAsource,origW,origN,low5,var_threshold,minmax,SVC,0.582624,0.129928,"{'kernel': 'rbf', 'C': 0.1, 'max_iter': 100, '...",0.551,0.171782,[ 0.3 0.56 0.6 0.55 0.3 0.55 0.9 0....
3,UCLAsource,origW,origN,low10,var_threshold,minmax,SVC,0.576418,0.162661,"{'kernel': 'poly', 'C': 0.3, 'max_iter': 100, ...",0.562,0.161852,[ 0.3 0.52 0.6 0.6 0.35 0.55 0.85 0....
4,UCLAsource,origW,origN,low20,var_threshold,minmax,SVC,0.582092,0.125035,"{'kernel': 'rbf', 'C': 0.1, 'max_iter': 100, '...",0.563,0.160253,[ 0.3 0.52 0.56 0.6 0.4 0.6 0.9 0....
5,UCLAsource,origW,origN,low30,var_threshold,minmax,SVC,0.595567,0.140763,"{'kernel': 'poly', 'C': 0.3, 'max_iter': 100, ...",0.583,0.144087,[ 0.4 0.52 0.56 0.6 0.5 0.6 0.9 0....
6,UCLAsource,origW,origN,low40,var_threshold,minmax,SVC,0.564716,0.141823,"{'kernel': 'sigmoid', 'C': 0.1, 'max_iter': 50...",0.557,0.164988,[ 0.3 0.56 0.56 0.7 0.45 0.6 0.9 0....
7,UCLAsource,origW,origN,low50,var_threshold,minmax,SVC,0.534397,0.116601,"{'kernel': 'linear', 'C': 0.001, 'max_iter': 5...",0.559,0.181022,[ 0.5 0.36 0.48 0.75 0.35 0.65 0.9 0....
8,UCLAsource,origW,origN,low60,var_threshold,minmax,SVC,0.553723,0.151243,"{'kernel': 'linear', 'C': 0.01, 'max_iter': 50...",0.564,0.176816,[ 0.4 0.44 0.6 0.75 0.4 0.6 0.95 0....
9,UCLAsource,origW,origN,low70,var_threshold,minmax,SVC,0.541667,0.130188,"{'kernel': 'linear', 'C': 0.01, 'max_iter': 50...",0.549667,0.168506,[ 0.46666667 0.36 0.52 0.8 ...


In [6]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig))]

normalizers = [('origN', Transformer(orig))]

featurizers = [('origF', Transformer(me.orig_vec, collect=['X_vec'])),
               ('low250', Transformer(me.matrix_eig_250, collect=['X_vec'])),
               ('low260', Transformer(me.matrix_eig_260, collect=['X_vec'])),
               ('low263', Transformer(me.matrix_eig_263, collect=['X_vec']))
              ]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler())]

classifiers = [('SVC_orig', SVC()),
               ('SVC_250', SVC()),
               ('SVC_260', SVC()),
               ('SVC_263', SVC()),
              ]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

param_grid = dict(
    LR_orig = dict(kernel= ['linear'], C= [0.01], max_iter= [100], degree= [2]),
    LR_250  = dict(kernel= ['rbf'], C= [0.01], max_iter= [50], degree= [2]),
    LR_260  = dict(kernel= ['linear'], C= [0.1], max_iter= [50], degree= [2]),
    LR_263  = dict(kernel= ['poly'], C= [0.1], max_iter= [50], degree= [4])
    )

banned_combos = []

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

In [7]:
k = [0, 5]+ [10*i for i in range(1, 27)] + [263]
col = result.as_matrix(["Featurizers"])
col_f = np.delete(np.hstack(col), 0)

for n,fet in enumerate(col_f):
    k_tmp = copy.copy(k)
    k_tmp.remove(k[n])
    for i in k_tmp: banned_combos.append((fet, 'SVC_' + str(i)))
        
for i in k: banned_combos.append(('origF', 'SVC_' + str(i)))
    
for fet in col_f: banned_combos.append((fet, 'SVC_orig'))

In [8]:
pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,origF,var_threshold,minmax,SVC_orig
1,UCLAsource,origW,origN,low250,var_threshold,minmax,SVC_250
2,UCLAsource,origW,origN,low260,var_threshold,minmax,SVC_260
3,UCLAsource,origW,origN,low263,var_threshold,minmax,SVC_263


In [9]:
num_epoch = 30

for epoch in range(2, num_epoch):

    grid_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=0)

    eval_cv = StratifiedKFold(n_splits=10,
                              shuffle=True,
                              random_state=epoch)

    
    
    pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
    #pipe.plan_table
    
    pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'],
                 scoring=['roc_auc'], results_file = "Results/Ver3.3/result_" + str(epoch) + ".csv")

No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: 4/4
No previous results found.
Line: 1/4
Line: 2/4
Line: 3/4
Line: